In [1]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
secret_value_1 = user_secrets.get_secret("NGROK_KEY")
model_chat = "meta-llama/Llama-2-13b-chat-hf"
tokenizer_chat = AutoTokenizer.from_pretrained(model_chat, token=secret_value_0)
pipeline_chat = transformers.pipeline(
    "text-generation",
    model=model_chat,
    torch_dtype=torch.float16,
    device_map="auto",
    token=secret_value_0,
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

2024-02-28 03:05:15.305534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 03:05:15.305644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 03:05:15.627181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [7]:
!pip install langchain
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline_chat)

In [8]:
from langchain import PromptTemplate
multi_input_template = """
<s>[INST] <<SYS>>
You are a helpful assistant that helps users generate questions. You will receive one question, and the question will be in JSON format. It will contain the question stem itself, no more than 4 options to answer the question, the correct option for the particular question, and the subject or topic related to the question.

You will do your best to generate one question, which should be similar in topic, style and difficulty of the question provided to you. It should be in JSON Format, and provide the question stem itself, the options, and the correct option to select.
<</SYS>>

{sample_in} [/INST] {sample_out} </s><s>[INST] {query} [/INST]
"""

multi_input_prompt = PromptTemplate(input_variables=["sample_in", "sample_out", "query"], template=multi_input_template)

In [9]:
sample_in="{'topic': 'Evolution', 'question': 'Man's evolution is defined as:', 'options': ['Dysversion', 'Anthropogenesis', 'Convergence', 'ontogenesis'], 'answer': 'B'}"
sample_out="{'question': 'Which of the following mechanisms is NOT a driving force of evolution?', 'options': ['Natural selection', 'Genetic drift', 'Lamarckism', 'Geographical barriers'], 'answer': 'C'}"
query="{'topic': 'Kinematics', 'question': 'Which of the following values is a vector?', 'options': ['Speed', 'Distance', 'Position', 'Velocity'], 'answer': 'D'}"
multi_input_prompt.format(sample_in=sample_in, sample_out=sample_out, query=query)
from langchain import LLMChain
llm_chain = LLMChain(prompt=multi_input_prompt, llm=llm)

In [10]:
!pip install flask
!pip install pyngrok

In [11]:
from flask import Flask, jsonify, request
import threading
from pyngrok import ngrok
import json
app = Flask(__name__)

@app.route('/run', methods=['GET'])
def hello_world():
    try:
        data_in = request.args.get('msg')
        data_out = llm_chain.invoke({"sample_in": sample_in, "sample_out": sample_out, "query": data_in})['text']
        return jsonify({"output": data_out})
    except Exception as e:
        print(e)

def run_flask():
    app.run(port=8000)

ngrok_auth_token = secret_value_1
ngrok.set_auth_token(ngrok_auth_token)
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
threading.Thread(target=run_flask).start()
print(f"Access your Flask app at: {public_url}")

Public URL: NgrokTunnel: "https://617f-34-134-248-15.ngrok-free.app" -> "http://localhost:8000"
Access your Flask app at: NgrokTunnel: "https://617f-34-134-248-15.ngrok-free.app" -> "http://localhost:8000"
 * Serving Flask app '__main__'
 * Debug mode: off


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentiall